<a href="https://colab.research.google.com/github/Aishashwini/Aishashwini/blob/main/Sentiment_Analysis_with_bert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**Project Information** The objectrive of this project is to detect the inappropriate tweets on twiter, such as racist or sexsit sentiment. Here lable 0 means not inappropreate tweet.

In [1]:
!pip install ktrain

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 25.3 MB 91.8 MB/s 
     |████████████████████████████████| 981 kB 49.5 MB/s 
     |████████████████████████████████| 263 kB 48.9 MB/s 
     |████████████████████████████████| 3.8 MB 23.8 MB/s 
     |████████████████████████████████| 1.3 MB 55.2 MB/s 
     |████████████████████████████████| 468 kB 53.3 MB/s 
     |████████████████████████████████| 880 kB 33.2 MB/s 
     |████████████████████████████████| 101 kB 10.7 MB/s 
     |████████████████████████████████| 596 kB 51.5 MB/s 
     |████████████████████████████████| 6.6 MB 30.1 MB/s 
  Created wheel for ktrain: filename=ktrain-0.31.7-py3-none-any.whl size=25312842 sha256=09e19319c8f275c75a53cd037f2e6ec18de515d231a8c232f8e1e4b99aeb56bd
  Stored in directory: /root/.cache/pip/wheels/e5/98/8e/ce355dcb92451e85fab93f7ea2da068843e93e703928cd06fb
  Created wheel for keras-bert: filename=keras_bert-0.89.0-

#**Import** **Libraries**

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import ktrain 
from ktrain import text
import seaborn as sns
import re
import string
import nltk
import warnings
warnings.filterwarnings('ignore')

#**Loading Data**

In [3]:
df = pd.read_csv('/content/Twitter Sentiments.csv')
df.head()

,id,label,tweet
0,1,0,@user when a father is dysfunctional and is s...
1,2,0,@user @user thanks for #lyft credit i can't us...
2,3,0,bihday your majesty
3,4,0,#model i love u take with u all the time in ...
4,5,0,factsguide: society now #motivation


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31962 entries, 0 to 31961
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      31962 non-null  int64 
 1   label   31962 non-null  int64 
 2   tweet   31962 non-null  object
dtypes: int64(2), object(1)
memory usage: 749.2+ KB


#**Preprocessing**

In [5]:
def remove_pattern(input_txt, pattern):
  r = re.findall(pattern, input_txt)
  for word in r:
    input_txt = re.sub(word, "", input_txt)
  return input_txt

In [6]:
# remove twitter handles (@users)
df['clean_tweet'] = np.vectorize(remove_pattern)(df['tweet'], "@[\w]*")
# remove special characters, numbers and punctuations
df['clean_tweet'] = df['clean_tweet'].str.replace("[^a-zA-Z#]", " ")
# remove short words
df['clean_tweet'] = df['clean_tweet'].apply(lambda x: " ".join([w for w in x.split() if len(w)>3]))
df.head()

,id,label,tweet,clean_tweet
0,1,0,@user when a father is dysfunctional and is s...,when father dysfunctional selfish drags kids i...
1,2,0,@user @user thanks for #lyft credit i can't us...,thanks #lyft credit cause they offer wheelchai...
2,3,0,bihday your majesty,bihday your majesty
3,4,0,#model i love u take with u all the time in ...,#model love take with time
4,5,0,factsguide: society now #motivation,factsguide society #motivation


#**Split Data**

In [7]:
df1=df[['clean_tweet','label']]
df1
from sklearn.model_selection import train_test_split
data_train, data_test = train_test_split(df1, random_state=42, test_size = 0.2)

In [8]:
data_train.head()

,clean_tweet,label
12110,because been paying attention past year years,1
14081,raft building salford quays outdoors badge #gu...,0
1829,friday #gdegblog #friday #selfie #beard #gaygu...,0
2769,#fashion true #fact,0
31818,#share #simple #elegant #businesscards #design...,0


In [9]:
data_test.head()

,clean_tweet,label
12227,says smile captivating says happy sunday pugsl...,0
14709,days will meeting couney bowers first lifestyl...,0
19319,hating conservative homophobes using this trag...,0
4308,awee this doesn #scream #friday #acewellstucke...,0
24055,fathersday #father #day #god #tony #smith thin...,0


In [10]:
(x_train, y_train), (x_test, y_test), preprocess =text.texts_from_df(train_df = data_train,
                   text_column= 'clean_tweet',
                   label_columns= 'label',
                   val_df= data_test,
                   maxlen = 100,
                   preprocess_mode= 'bert')

['not_label', 'label']
       not_label  label
12110        0.0    1.0
14081        1.0    0.0
1829         1.0    0.0
2769         1.0    0.0
31818        1.0    0.0
['not_label', 'label']
       not_label  label
12227        1.0    0.0
14709        1.0    0.0
19319        1.0    0.0
4308         1.0    0.0
24055        1.0    0.0
downloading pretrained BERT model (uncased_L-12_H-768_A-12.zip)...
[██████████████████████████████████████████████████]
extracting pretrained BERT model...
done.

cleanup downloaded zip...
done.

preprocessing train...
language: en


Is Multi-Label? False
preprocessing test...
language: en


In [11]:
model = text.text_classifier(name = 'bert',
                             train_data = (x_train, y_train),
                             preproc = preprocess)

Is Multi-Label? False
maxlen is 100
done.


In [12]:
# get the learning rate
learner = ktrain.get_learner(model = model,
                             train_data = (x_train, y_train),
                             val_data = (x_test, y_test),
                             batch_size = 32)

In [13]:
# this takes almost 1 to 2 days to run
# learner.lr_find()
# learner.lr_plot()
# optimal learning rate for this model is 3e-5

In [14]:
learner.fit_onecycle(lr = 3e-5, epochs = 1)



begin training using onecycle policy with max lr of 3e-05...
800/800 [==============================] - 31249s 39s/step - loss: 0.1595 - accuracy: 0.9464 - val_loss: 0.1021 - val_accuracy: 0.9632


In [15]:
predictor = ktrain.get_predictor(learner.model, preprocess)

In [18]:
predictor.save('/bert')